In [48]:
import agate

In [49]:
exonerations = agate.Table.from_csv('exonerations-20150828.csv')

print exonerations


|----------------------+---------------|
|  column_names        | column_types  |
|----------------------+---------------|
|  last_name           | Text          |
|  first_name          | Text          |
|  age                 | Number        |
|  race                | Text          |
|  state               | Text          |
|  tags                | Text          |
|  crime               | Text          |
|  sentence            | Text          |
|  convicted           | Number        |
|  exonerated          | Number        |
|  dna                 | Boolean       |
|  dna_essential       | Text          |
|  mistake_witness     | Boolean       |
|  false_confession    | Boolean       |
|  perjury             | Boolean       |
|  false_evidence      | Boolean       |
|  official_misconduct | Boolean       |
|  inadequate_defense  | Boolean       |
|----------------------+---------------|



In [50]:
print exonerations.columns[10]

<agate.Column: (True, True, True, None, None, ...)>


In [51]:
first_row = exonerations.rows[0]
print first_row
print first_row['dna']

<agate.Row: (u'Abbitt', u'Joseph Lamont', Decimal('31'), u'Black', u'NC', ...)>
True


In [52]:
num_false_confessions = exonerations.aggregate(agate.Count('false_confession', True))

print(num_false_confessions)

211


In [53]:
median_age = exonerations.aggregate(agate.Median('age'))

print(median_age)

26


In [54]:
num_without_age = exonerations.aggregate(agate.Count('age', None))

print(num_without_age)

9


In [55]:
mo = exonerations.aggregate(agate.Count('state', 'MO'))
print mo

34


In [56]:
with_age = exonerations.where(lambda row: row['age'] is not None)

In [57]:
old = len(exonerations.rows)
new = len(with_age.rows)

print(old - new)

9


In [58]:
median_age = with_age.aggregate(agate.Median('age'))

print(median_age)

26


In [59]:
with_years_in_prison = exonerations.compute([
    ('years_in_prison', agate.Change('convicted', 'exonerated'))
])

median_years = with_years_in_prison.aggregate(agate.Median('years_in_prison'))

print(median_years)

8


In [60]:
full_names = exonerations.compute([
    ('full_name', agate.Formula(agate.Text(), lambda row: '%(first_name)s %(last_name)s' % row))
])

In [61]:
with_computations = exonerations.compute([
    ('full_name', agate.Formula(agate.Text(), lambda row: '%(first_name)s %(last_name)s' % row)),
    ('years_in_prison', agate.Change('convicted', 'exonerated'))
])

In [62]:
sorted_by_age = exonerations.order_by('age')

youngest_ten = sorted_by_age.limit(10)

youngest_ten.print_table(max_columns=7)


|------------+------------+-----+-----------+-------+---------+---------+------|
|  last_name | first_name | age | race      | state | tags    | crime   | ...  |
|------------+------------+-----+-----------+-------+---------+---------+------|
|  Murray    | Lacresha   |  11 | Black     | TX    | CV, F   | Murder  | ...  |
|  Adams     | Johnathan  |  12 | Caucasian | GA    | CV, P   | Murder  | ...  |
|  Harris    | Anthony    |  12 | Black     | OH    | CV      | Murder  | ...  |
|  Edmonds   | Tyler      |  13 | Caucasian | MS    |         | Murder  | ...  |
|  Handley   | Zachary    |  13 | Caucasian | PA    | A, CV   | Arson   | ...  |
|  Jimenez   | Thaddeus   |  13 | Hispanic  | IL    |         | Murder  | ...  |
|  Pacek     | Jerry      |  13 | Caucasian | PA    |         | Murder  | ...  |
|  Barr      | Jonathan   |  14 | Black     | IL    | CDC, CV | Murder  | ...  |
|  Brim      | Dominique  |  14 | Black     | MI    | F       | Assault | ...  |
|  Brown     | Timothy    | 

In [63]:
binned_ages = exonerations.bins('age', 10, 0, 100)
binned_ages.print_bars('age', 'Count', width=80)

age       Count
[10 - 20)   307 ▓░░░░░░░░░░░░░░░░░░░░░░░░                                       
[20 - 30)   718 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░      
[30 - 40)   377 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                                 
[40 - 50)   176 ▓░░░░░░░░░░░░░░                                                 
[50 - 60)    53 ▓░░░░                                                           
[60 - 70)    10 ▓░                                                              
[80 - 90)     1 ▓                                                               
None          9 ▓░                                                              
                +---------------+---------------+--------------+---------------+
                0              200             400            600            800


In [64]:
by_state = exonerations.group_by('state')

In [65]:
state_totals = by_state.aggregate([
        ('count', agate.Length())
])

sorted_totals = state_totals.order_by('count', reverse=True)

sorted_totals.print_table(max_rows=5)

AttributeError: 'module' object has no attribute 'Length'

In [ ]:
with_years_in_prison = exonerations.compute([
    ('years_in_prison', agate.Change('convicted', 'exonerated'))
])

state_totals = with_years_in_prison.group_by('state')

medians = state_totals.aggregate([
    ('count', agate.Length()),
    ('median_years_in_prison', agate.Median('years_in_prison'))
])

sorted_medians = medians.order_by('median_years_in_prison', reverse=True)

sorted_medians.print_table(max_rows=5)

In [ ]:
# Filters rows without age data
only_with_age = with_years_in_prison.where(
    lambda r: r['age'] is not None
)

# Group by race
race_groups = only_with_age.group_by('race')

# Sub-group by age cohorts (20s, 30s, etc.)
race_and_age_groups = race_groups.group_by(
    lambda r: '%i0s' % (r['age'] // 10),
    key_name='age_group'
)

# Aggregate medians for each group
medians = race_and_age_groups.aggregate([
    ('count', agate.Length()),
    ('median_years_in_prison', agate.Median('years_in_prison'))
])

# Sort the results
sorted_groups = medians.order_by('median_years_in_prison', reverse=True)

# Print out the results
sorted_groups.print_table(max_rows=10)